<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/04_%E7%89%A9%E4%BD%93%E8%AA%8D%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体認識

## 環境設定

## ライブラリインストール

In [1]:
!pip install open3d==0.16

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.5/422.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.8.0
    Uninstalling nbformat-5.8.0:
      Successfully uninstalled nbformat-5.8.0


## 物体認識の概要

## 物体認識

### データのダウンロード

In [2]:
import os
import open3d as o3d
import numpy as np

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")



### 表示して確認

In [3]:
apple = "/content/rgbd-dataset/apple/apple_1/apple_1_1_3.pcd"
# main

print("Loading a point cloud from", apple)
pcd = o3d.io.read_point_cloud(apple)
print(pcd)


pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [pcd],
  width=1200,
  height=800
)


Loading a point cloud from /content/rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
PointCloud with 3198 points.


### 特徴量の算出

In [4]:
def extract_fpfh( filename ):
    print (" ", filename)
    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(0.01)
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )

# Extract features FPFH
nsamp = 100
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )
for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )




Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

### 分類器で推論

今回は機械学習モデルを使用した学習は時間がかかるので、作成した特徴量との類似度を計算して分類します。


In [5]:
# 1-NN classification
for i in range(len(classes)):
    max_sim = np.zeros((3, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose()) # 内積を計算して
        max_sim[j] = np.max(sim,1) # 一番近い特徴量を探す
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of apple : 98.0 %
Accuracy of banana : 89.0 %
Accuracy of camera : 83.0 %


## より多くのクラスで分類


In [6]:
import os
import open3d as o3d
import numpy as np

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera", "cell_phone", "food_bag", "lemon", "scissors", "sponge", "potato" , "keyboard"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")



In [7]:
def extract_fpfh( filename ):
    print (" ", filename)
    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(0.01)
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )

# Extract features FPFH
nsamp = 100
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )
for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )




Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

In [9]:
# 1-NN classification
for i in range(len(classes)):
    max_sim = np.zeros((10, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose()) # 内積を計算して
        max_sim[j] = np.max(sim,1) # 一番近い特徴量を探す
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of apple : 42.0 %
Accuracy of banana : 73.0 %
Accuracy of camera : 54.0 %
Accuracy of cell_phone : 20.0 %
Accuracy of food_bag : 70.0 %
Accuracy of lemon : 67.0 %
Accuracy of scissors : 17.0 %
Accuracy of sponge : 5.0 %
Accuracy of potato : 93.0 %
Accuracy of keyboard : 0.0 %


3クラスのときよりも分類性能が悪くなったことが確認できます。
そのため、特徴量の作成では限界があると言えます。
したがって、深層学習のようなデータから特徴を獲得するような手法が有効ではないかと考えられます。
